In [9]:
import numpy as np
import random
import math

## seed the random number generator for all but the initial condtition x
rng1 = np.random.default_rng(seed=0)

def move(x, D, rng):
    """Move the state vector ."""
    u = rng.uniform(-1, 1, len(x))
    alpha = 0.1
    w = 2.1
    R = np.diag(np.dot(D, u))
    D = (1 - alpha) * D + alpha * w * R
    min_step_size = 1e-4 #FIX
    D = np.where(D < min_step_size, min_step_size, D)
    x = x + np.dot(D, u)
    return x, D, u

def objective(x, n):
    """Keane's Bump function.
    n = dimension of the domain
    x = vector in the domain"""
    cos4 = 0
    cos2 = 1
    x_i = 0
    for i in range(n):
        cos4 += math.cos(x[i])**4
        cos2 *= math.cos(x[i])**2
        x_i += (i+1)*(x[i]**2)
    if x_i == 0:
        answer = None
    else:
        answer = np.abs((cos4 - 2*cos2)/math.sqrt(x_i))
    ## we are turning the maximization problem into a minimization problem
    return -1 * answer

def assess_solution(x_prev, n, D_prev, T, counter, rng):
    f_prev = objective(x_prev, n)
    x, D, u = move(x_prev, D_prev, rng)
    ## test constraints and reject move, if necessary
    if np.all(x) < 0 or np.all(x) > 10:
        return x_prev, D_prev, counter
    if np.prod(x) <= 0.75 or np.sum(x) >= (15*n/2):
        return x_prev, D_prev, counter
    f = objective(x, n)
    if f < f_prev:
        counter += 1
        return x, D, counter
    else:
        step_size = np.sqrt(np.sum(np.dot(D, u)**2))
        acceptance_prob = np.exp((f_prev - f)/(T*step_size))
        if random.random() < acceptance_prob:
            counter += 1
            return x, D, counter
        else:
            counter += 1
            return x_prev, D_prev, counter

def temperature_schedule(T, alpha_T):
    return alpha_T*T

def archive_function(x, f, archive):
    ## 10 best solutions 
    archive_limit = 10
    if len(archive) < archive_limit:
        archive.append((x, f))
    else:
        archive.append((x, f))  # Update all elements
        archive.sort(key=lambda item: item[1])  # Sort the archive by objective function value
        archive = archive[:archive_limit]  # Keep only the top elements
    return archive


def main(x, n, Tin):
    ## initialize covariance matrix
    D = 2 * np.identity(n)
    ## initial temperature (FOUND THROUGH EMPIRCAL TUNING)
    T = Tin
    ## final temperature (FIX)
    T_min = 1e-5
    ## min acceptance ratio (HYPER PARAMETER)
    min_acceptance_ratio = 0.01
    ## cooling rate (HYPER PARAMETER)
    alpha_T = 0.95
    ## markov chain length (HYPER PARAMETER)
    Lk = 500
    nmin = Lk *0.6

    ## initailize state, covariance matrix
    x_prev = x
    D_prev = D
    ## max number of iterations
    # max_iter = 10000
    counter = 0
    ## initialize archive
    archive = []

    accepted_points = []

    while T > T_min:
    # while True:
        acceptances = 0
        ## when either Lk trials or nmin acceptances whichever comes first, decrement the temperature 
        for i in range(Lk):
            x, D, counter = assess_solution(x_prev, n, D_prev, T, counter, rng1)
            if not np.array_equal(x, x_prev):
                acceptances += 1
                archive = archive_function(x, objective(x, n), archive)
                accepted_points.append(x)
                x_prev = x
                D_prev = D
            if acceptances == nmin:
                break
        ## decrement temperature
        T = temperature_schedule(T, alpha_T)
        ## no accepted moves in Lk iterations at temperature Tk
        if acceptances/ Lk <= min_acceptance_ratio:
            print(counter)
            break
        # if counter >= max_iter:
        #     print(f"counter == max_iter, break")
        #     break
    return archive, accepted_points

In [11]:
while True:
    x = np.round(np.random.uniform(0, 10, 8), 2)
    n = 8
    if np.prod(x) >= 0.75 and np.sum(x) <= (15*n/2):
        break
print(x)

[2.34 9.74 9.6  2.82 3.7  9.17 7.68 3.14]


In [12]:
archive, accepted_points = main(x, n, Tin = 0.2)

In [13]:
best = min(archive, key=lambda x: x[1])
print(best)

(array([ 3.01800499, 10.513889  ,  9.22421438,  0.51428127,  6.54118185,
        9.53216532,  8.94777199,  3.06737619]), -0.1326266187959923)


Now that we have confirmed that the 8D case is working as expected, we can fine tune to find the performance when tuning different hyper-parameters